In [41]:
import tkinter as tk
import pandas as pd

filename = 'new_sampled_train_set_seg2_3.csv'
df = pd.read_csv(filename)

In [42]:
class LabelingApp:
    def __init__(self, root, df, csv_file):
        self.root = root
        self.root.title("감정 레이블링 도구")
        
        # 창 크기 설정
        self.root.geometry("1800x1000")  # 창 크기를 가로 1800, 세로 1000으로 설정
        
        self.df = df
        self.csv_file = csv_file  # csv_file 인자를 사용하여 파일 경로 저장
        self.index = 0
        
        # sentiment 열이 비어있는 행부터 시작
        self.index = self.find_first_empty_sentiment()
        
        # Group, Title, 댓글을 보여주는 라벨
        self.group_label = tk.Label(self.root, text="", font=("Arial", 30, "bold"), fg="gray")
        self.group_label.pack(pady=(5, 0))  # 줄 간격 줄이기
        
        self.title_label = tk.Label(self.root, text="", font=("Arial", 30, "bold"), fg="gray")
        self.title_label.pack(pady=(0, 5))  # 줄 간격 줄이기
        
        self.comment_label = tk.Label(self.root, text="", wraplength=600, height=5, font=("Arial", 30))
        self.comment_label.pack(pady=(0, 10))  # 줄 간격 줄이기
        
        # sentiment를 보여주는 라벨
        self.sentiment_info = tk.Label(self.root, text="Sentiment: ", font=("Arial", 30))
        self.sentiment_info.pack(pady=10)
        
        # 레이블링 현황을 보여주는 라벨
        self.count_label = tk.Label(self.root, text="", font=("Arial", 20))
        self.count_label.pack(pady=10)
        
        # list 버튼 추가
        self.list_button = tk.Button(self.root, text="List", command=self.show_list, font=("Arial", 20))
        self.list_button.pack(pady=10)

        # 종료 버튼 추가
        self.quit_button = tk.Button(self.root, text="종료", command=self.quit_program, font=("Arial", 20))
        self.quit_button.pack(pady=10)
        
        # Listbox 추가
        self.listbox = tk.Listbox(self.root, height=15, width=80, font=("Arial", 20))
        self.listbox.pack(pady=10)
        self.listbox.bind("<<ListboxSelect>>", self.on_select)

        # 키 바인딩
        self.root.bind("<Down>", self.sentiment_as_neutral)   # Neutral (1)
        self.root.bind("<Right>", self.sentiment_as_positive)  # Positive (2)
        self.root.bind("<Left>", self.sentiment_as_negative) # Negative (0)
        self.root.bind("<Shift-Left>", self.prev_comment)     # Shift + Left: 이전 댓글
        self.root.bind("<Shift-Right>", self.next_comment)    # Shift + Right: 다음 댓글
        
        # 처음 댓글 표시
        self.display_comment()

        # count_label 업데이트 (초기값)
        self.update_count_label()

    # sentiment가 비어있는 첫 번째 인덱스를 찾는 함수
    def find_first_empty_sentiment(self):
        for i, sentiment in enumerate(self.df['sentiment']):
            if pd.isna(sentiment) or sentiment == "":
                return i
        return 0  # 모든 sentiment가 채워져 있다면 첫 번째 행부터 시작

    def display_comment(self):
        group = self.df.loc[self.index, 'Group']
        title = self.df.loc[self.index, 'Title']
        comment = self.df.loc[self.index, 'comment']
        
        # Group, Title, 댓글 정보 표시
        self.group_label.config(text=f"Group: {group}")
        self.title_label.config(text=f"Title: {title}")
        self.comment_label.config(text=comment)
        
        # 감정 레이블 표시
        if 'sentiment' in self.df.columns and not pd.isna(self.df.loc[self.index, 'sentiment']):
            current_sentiment = self.df.loc[self.index, 'sentiment']
            self.sentiment_info.config(text=f"Sentiment: {current_sentiment}")
        else:
            self.sentiment_info.config(text="Sentiment: ")
        
        # 리스트 갱신
        self.update_listbox()

    def sentiment_as_neutral(self, event):
        self.save_sentiment("1")
        
    def sentiment_as_positive(self, event):
        self.save_sentiment("2")
        
    def sentiment_as_negative(self, event):
        self.save_sentiment("0")
        
    def save_sentiment(self, sentiment):
        self.df.loc[self.index, 'sentiment'] = str(sentiment)  # 타입을 str로 설정
        self.save_to_csv()  # CSV 저장
        self.next_comment(None)
        
    def prev_comment(self, event):
        if self.index > 0:
            self.index -= 1
        self.display_comment()
        
    def next_comment(self, event):
        if self.index < len(self.df) - 1:
            self.index += 1
        self.display_comment()

    def quit_program(self):
        self.root.quit()
        self.root.destroy()

    # CSV로 저장하는 함수
    def save_to_csv(self):
        self.df.to_csv(self.csv_file, index=False)  # self.filepath 대신 self.csv_file 사용
        print(f"CSV 저장 완료: {self.csv_file}")
        # 레이블링 현황 업데이트
        self.update_count_label()

    # count_label 업데이트 함수
    def update_count_label(self):
        count = f"{len(self.df) - sum(self.df['sentiment'].isna())} / {len(self.df)}"
        self.count_label.config(text=f"Labeled: {count}")
        print(count)

    # List 버튼 눌렀을 때 10개씩 댓글을 보여줌
    def show_list(self):
        self.update_listbox()

    # Listbox 업데이트 (앞뒤로 10개씩 보여줌)
    def update_listbox(self):
        self.listbox.delete(0, tk.END)
        start = max(0, self.index - 10)
        end = min(len(self.df), self.index + 10)
        
        for i in range(start, end):
            comment = self.df.loc[i, 'comment']
            sentiment = self.df.loc[i, 'sentiment']
            if i == self.index:
                self.listbox.insert(tk.END, f"→ {comment}")  # 현재 댓글을 화살표로 강조
                self.listbox.itemconfig(tk.END, {'bg': 'gray'})  # 현재 댓글의 배경색을 회색으로 설정
            else:
                self.listbox.insert(tk.END, comment)
                # sentiment가 비어있지 않으면 옅은 회색 배경 적용
                if pd.notna(sentiment) and sentiment != "":
                    self.listbox.itemconfig(tk.END, {'bg': '#d3d3d3'})  # 옅은 회색 배경

    # Listbox에서 댓글 선택 시
    def on_select(self, event):
        try:
            selection = event.widget.curselection()
            if selection:
                self.index = selection[0] + max(0, self.index - 10)  # Listbox에서 선택한 인덱스를 반영
                self.display_comment()
        except Exception as e:
            pass


# GUI 실행
root = tk.Tk()
app = LabelingApp(root, df, filename)
root.mainloop()


0 / 120


/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_73321/2575272723.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df.loc[self.index, 'sentiment'] = str(sentiment)  # 타입을 str로 설정


CSV 저장 완료: new_sampled_train_set_seg2_3.csv
1 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
2 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
3 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
4 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
5 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
6 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
7 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
8 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
9 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
10 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
11 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
12 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
13 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
13 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
14 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
15 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
15 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
16 / 120
CSV 저장 완료: new_sampled_train_set_seg2_3.csv
17 / 120
CS

In [43]:
df2 = pd.read_csv(filename)
df2[-30:]

,Index,Group,Title,Unnamed: 0,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,sentiment
90,541775,NewJeans,Ditto (side A),2130,reply,NaN,UgzKMY8mZgSy_yb51NZ4AaABAg,반희수 = 버니즈 = 뉴진스 팬덤명,@박진엽-b6g,2024-04-30T16:34:32Z,0,1
91,2390,NMIXX,별별별 (See that),16595,node,UgwWzFb0iBUqRtfiXeV4AaABAg,NaN,오렌지 먹은지 얼마나 오렌지,@오렌지먹은지얼마나오,2024-08-19T13:50:27Z,1,1
92,167005,IVE,After LIKE,174276,node,UgxHSf8DTvWPIT_8OPZ4AaABAg,NaN,솔직히 이 노래는 보컬없는 inst가 찐이다...,@Geugler,2022-08-22T09:24:18Z,1,1
93,108879,ITZY,마.피.아. In the morning,246077,node,UgwfeintvHrgPPYFppR4AaABAg,NaN,솔직히 존나 박진영이 너무했잖아 이게뭐냐고,@치즈크로스토,2021-05-05T11:07:44Z,0,0
94,79167,FIFTY FIFTY,Cupid,506,node,UgyJFNU8yllxR3jrbkl4AaABAg,NaN,첫소절 부르는 친구 음색 진짜 너무 아깝다.....너무 좋은데 에휴,@user-nn3zr9rb8s,2024-09-22T16:33:23Z,3,0
95,142194,ITZY,WANNABE,266192,reply,NaN,Ugwoodp2sqQpQo_YYfN4AaABAg,헐 진저님 안녕하세요!!!,@kellylee421,2020-03-09T15:06:21Z,0,1
96,62900,FIFTY FIFTY,Starry Night,6479,reply,NaN,UgxydMJAesG2NOaSWY94AaABAg,함께 달려봅시다.~~️,@내가빅팬,2024-09-06T22:38:35Z,1,2
97,484236,NewJeans,OMG,39115,node,UgyBhPMllUFHE_WraZF4AaABAg,NaN,여러분 구글 gemini(제미나이)에 식인의 연관성 쳐보면 알려주네요,@Dole-fn9lx,2024-04-29T08:56:06Z,11,1
98,526997,NewJeans,OMG,98557,reply,NaN,UgwJLiEMvibLDeXjJOR4AaABAg,제발 생각이란걸 하고 살아.... 뇌피셜밖에 없는 내용을 오피셜인 기사로 어캐 씀;;;;,@Doggut,2024-04-28T05:42:23Z,0,0
99,388898,aespa,Savage,202152,node,Ugy1IHzpUF5Ac8zZO714AaABAg,NaN,어스파,@blueberrysmoothie6063,2021-10-06T15:50:12Z,0,1


In [4]:
import pandas as pd

df1 = pd.read_csv('12000/learning_set_4000(1).csv')
df2 = pd.read_csv('12000/learning_set_4000(2).csv')
df3 = pd.read_csv('12000/learning_set_4000(3).csv')

df_full = pd.concat([df1, df2, df3], axis=0)

In [5]:
# 0, 3, 12, 13 열 드랍 (순서로)
print(df_full.columns[[0,3,12,13]])
df_full_clean = df_full.drop(df_full.columns[[0, 3, 12, 13]], axis=1)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 12', 'Index'], dtype='object')


In [7]:
df_full_clean.to_csv('12000/learning_set_12000.csv', encoding='utf-8-sig', index=False)

In [22]:
from sklearn.model_selection import train_test_split

# 10000개 샘플 추출
df_full_clean_train, remaining_df = train_test_split(df_full_clean, train_size=10000, random_state=42)

# 나머지 데이터프레임에서 1000개 샘플 추출
df_full_clean_valid, df_full_clean_test = train_test_split(remaining_df, train_size=1000, random_state=42)

print(len(df_full_clean_train))
print(len(df_full_clean_valid))
print(len(df_full_clean_test))

df_full_clean_train.to_csv('12000/learning_set_train_10000.csv', encoding='utf-8-sig', index=False)
df_full_clean_train.to_csv('12000/learning_set_valid_1000.csv', encoding='utf-8-sig', index=False)
df_full_clean_train.to_csv('12000/learning_set_test_1000.csv', encoding='utf-8-sig', index=False)

10000
1000
1000


In [14]:
df_full_clean_train['sentiment'].value_counts()

sentiment
2    5778
0    2395
1    1827
Name: count, dtype: int64

In [20]:
import pandas as pd
from sklearn.utils import resample

# 예시 데이터프레임 (sentiment 레이블과 그 외 feature들로 구성된 데이터프레임)
# df = pd.read_csv('your_data.csv') 또는 적절한 데이터 로드 방식 사용

# 'sentiment'가 0, 1인 데이터프레임 필터링
df_0 = df_full_clean_train[df_full_clean_train['sentiment'] == 0]
df_1 = df_full_clean_train[df_full_clean_train['sentiment'] == 1]
df_2 = df_full_clean_train[df_full_clean_train['sentiment'] == 2]  # 클래스 2는 이미 6963개이므로 그대로 사용

# 각 클래스를 6963개로 맞추기 위해 오버샘플링
df_0_oversampled = resample(df_0, replace=True, n_samples=len(df_2), random_state=42)
df_1_oversampled = resample(df_1, replace=True, n_samples=len(df_2), random_state=42)

# 오버샘플링된 데이터와 기존 클래스 2 데이터를 합침
df_balanced = pd.concat([df_0_oversampled, df_1_oversampled, df_2])

# 데이터프레임을 섞어줌 (optional)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_balanced = df_balanced.sample(n=17000, random_state=42)

# 결과 확인
print(df_balanced['sentiment'].value_counts())


sentiment
2    5677
1    5674
0    5649
Name: count, dtype: int64


In [23]:
df_balanced.to_csv('12000/learning_set_train_17000_oversampled.csv', encoding='utf-8-sig', index=False)